In [47]:
%cd "/content/drive/MyDrive/MIR datasets"
!git clone https://github.com/hooshvare/parsbert.git
%cd parsbert
!pip install -r requirements.txt
!pip install transformers
!pip install numpy
!pip install hazm

/content/drive/MyDrive/MIR datasets
fatal: destination path 'parsbert' already exists and is not an empty directory.
/content/drive/MyDrive/MIR datasets/parsbert
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 884 kB 7.5 MB/s 
     |████████████████████████████████| 748.8 MB 18 kB/s 
     |████████████████████████████████| 6.8 MB 44.6 MB/s 
ERROR: Could not find a version that satisfies the requirement dataclasses~=0.7 (from versions: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6)
ERROR: No matching distribution found for dataclasses~=0.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 32.6 MB/s 
     |████████████████████████████████| 6.6 MB 52.0 MB/s 
     |████████████████████████████████| 596 kB 66.9 MB/s 
     |████████████████████████████████| 101 kB 7.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing i

In [48]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

# load model and tokenizer
# this will download the model for you if you don't have it which is around 500MB
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
pmodel = AutoModel.from_pretrained(model_name_or_path, output_hidden_states=True)
pmodel.eval()

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [5]:
import torch
import pickle as pkl
with_stopwords_filepath = "/content/drive/MyDrive/MIR datasets/embeddings_dataset_with_stopwords.pkl"
# link to drive = https://drive.google.com/file/d/1uixWaOOTW0oSfu5mx4-qZmo7__wzX1QA/view?usp=sharing

with open(with_stopwords_filepath, 'rb') as f:
  loaded_pickle = pkl.load(f)
  embeddings_with, original_beyts = loaded_pickle['embeddings'], loaded_pickle['beyts']

# make a mapping of beyts to embeddings to make a dataset easier
mapping = {}
for i,beyt in enumerate(original_beyts):
  mapping[beyt] = embeddings_with[i]

In [53]:
def get_sent_embedding(text):
  tokenized_text = tokenizer.tokenize(text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  token_tensors = torch.tensor([indexed_tokens]).cuda()
  out = pmodel(token_tensors)
  return out[2][-2].mean(1).squeeze()

In [26]:
filepath = "/content/drive/MyDrive/MIR datasets/preprocessed_doc.pkl"
# drive link to file = https://drive.google.com/file/d/14DtgrwLQLO-PtMKN4cvxIlIQIFYf-C5C/view?usp=sharing 

with open(filepath, "rb") as f:
    shahname_all = pkl.load(f)


classes = {'داستان دوازده رخ': 0,
            'داستان اکوان دیو': 1,
           'داستان رستم و اسفندیار': 2,
           'داستان سیاوش': 3}
class_weights = [0, 0, 0, 0]

data = []

for k in shahname_all:
  if k[1] in classes:
    for beyt in shahname_all[k]:
      class_weights[classes[k[1]]] += 1
      data.append((mapping[beyt[1]], classes[k[1]]))

In [99]:
class ShahnameDataset(torch.utils.data.Dataset):
  def __init__(self, s_data):
    self.data = s_data

  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, idx):
    return data[idx]


ds = ShahnameDataset(data)
dataloader = torch.utils.data.DataLoader(ds, batch_size=32,
                        shuffle=True, num_workers=0)

In [114]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Classifier(nn.Module):
  def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(768, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 254)
        self.fc4 = nn.Linear(254, 64)
        self.fc5 = nn.Linear(64, 4)

  def forward(self, x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = F.relu(self.fc3(x))
      x = F.relu(self.fc4(x))
      x = self.fc5(x)
      return x

model = Classifier().cuda()
criterion = nn.CrossEntropyLoss()#weight = torch.tensor(class_weights).float().cuda())
optimizer = optim.Adam(model.parameters(), weight_decay = 0.0001)

In [115]:
num_epochs = 20

for epoch in range(num_epochs):
  acc = 0
  loss = 0
  n_points = 0
  if epoch == 15:
    optimizer.param_groups[0].update(lr=1e-4)
  for i, (X, y) in enumerate(dataloader):
    y = y.cuda()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    acc += (outputs.max(1)[1] == y).sum().item()
    loss += loss.item()
    n_points += X.size(0)
  print("Epoch: {:d} --- Acc: {:.4f} ## Loss: {:.4f}".format(epoch, acc/n_points, loss/n_points))

Epoch: 0 --- Acc: 0.5230 ## Loss: 0.0003
Epoch: 1 --- Acc: 0.5641 ## Loss: 0.0002
Epoch: 2 --- Acc: 0.5792 ## Loss: 0.0005
Epoch: 3 --- Acc: 0.5901 ## Loss: 0.0002
Epoch: 4 --- Acc: 0.6180 ## Loss: 0.0002
Epoch: 5 --- Acc: 0.6479 ## Loss: 0.0001
Epoch: 6 --- Acc: 0.6760 ## Loss: 0.0001
Epoch: 7 --- Acc: 0.7090 ## Loss: 0.0001
Epoch: 8 --- Acc: 0.7418 ## Loss: 0.0001
Epoch: 9 --- Acc: 0.7778 ## Loss: 0.0003
Epoch: 10 --- Acc: 0.7983 ## Loss: 0.0001
Epoch: 11 --- Acc: 0.8363 ## Loss: 0.0001
Epoch: 12 --- Acc: 0.8701 ## Loss: 0.0001
Epoch: 13 --- Acc: 0.8897 ## Loss: 0.0002
Epoch: 14 --- Acc: 0.9057 ## Loss: 0.0000
Epoch: 15 --- Acc: 0.9628 ## Loss: 0.0000
Epoch: 16 --- Acc: 0.9804 ## Loss: 0.0000
Epoch: 17 --- Acc: 0.9870 ## Loss: 0.0000
Epoch: 18 --- Acc: 0.9902 ## Loss: 0.0000
Epoch: 19 --- Acc: 0.9936 ## Loss: 0.0000


In [116]:
# import our own preprocessing (make sure you have get_document.py from the project files present)
from get_document import preprocess_sent

model.eval()
pmodel.cuda()
reverse_classes = {}
for c in classes:
  reverse_classes[classes[c]] = c

def respond_to_query(text):
  processed_q = ' '.join(preprocess_sent(text))
  q_embedding = get_sent_embedding(processed_q).unsqueeze(0)
  return reverse_classes[(model(q_embedding).squeeze().argmax().item())]

queries = [
           'نهادند بر پشتشان تخت زر نشستنگه شاه با زیب و فر',
           'گر افراسیاب این سخنها که گفت به پیمان شکستن بخواهد نهفت',
           'ز بهمن برآشفت اسفندیار ورا بر سر انجمن کرد خوار',
           'ز رستم چو بشنید اکوان دیو برآورد بر سوی دریا غریو'
]

In [117]:
for query in queries:
  print(query, " ----- ", respond_to_query(query))

نهادند بر پشتشان تخت زر نشستنگه شاه با زیب و فر  -----  داستان دوازده رخ
گر افراسیاب این سخنها که گفت به پیمان شکستن بخواهد نهفت  -----  داستان سیاوش
ز بهمن برآشفت اسفندیار ورا بر سر انجمن کرد خوار  -----  داستان رستم و اسفندیار
ز رستم چو بشنید اکوان دیو برآورد بر سوی دریا غریو  -----  داستان اکوان دیو


In [ ]:
respond_to_query('')